In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchtsmixer import TSMixer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from src.training import train_tsmixer,train_latent_tsmixer,run_trials

#add a path to the system path
import sys
sys.path.append('../')

from experiments.datasets import *
seed=42
torch.manual_seed(seed)

# AutoRegresive experiments

## Synthetic

In [2]:
sequence_length = 40
prediction_length = 20
input_channels = 1
output_channels = 1
n_samples=200
n_timesteps=sequence_length+prediction_length

#generate a Sine dataset
Dataset_generator=SineTransDataset_AR(n_samples=n_samples, n_timesteps=n_timesteps)
X,ts,ys=Dataset_generator.get_X_ts_ys()

Ys=np.array(ys)
X_torch=torch.tensor(Ys[:,:sequence_length]).float().reshape(n_samples,sequence_length,1)
Y_torch=torch.tensor(Ys[:,sequence_length:]).float().reshape(n_samples,prediction_length,1)
print(X_torch.shape,Y_torch.shape)

torch.Size([200, 40, 1]) torch.Size([200, 20, 1])


In [3]:
model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

100%|██████████| 10/10 [00:25<00:00,  2.54s/it]

Mean Test Loss: 0.0032, Std Test Loss: 0.0048


In [4]:
from timeview.basis import BSplineBasis
from timeview.knot_selection import calculate_knot_placement
B=5
ts_N=[arr[sequence_length:] for arr in ts]
ys_N=[arr[sequence_length:] for arr in ys]

internal_knots=calculate_knot_placement(ts_N, ys_N, n_internal_knots=B-2, T=ts_N[0][-1], Ti=ts_N[0][0] ,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(ts_N[0][0],ts_N[0][-1]),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(ts)[:,sequence_length:]))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)



mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
100%|██████████| 10/10 [01:02<00:00,  6.23s/it]

Mean Test Loss: 0.0034, Std Test Loss: 0.0050


In [5]:
sequence_length = 40
prediction_length = 20
input_channels = 1
output_channels = 1
n_samples=900
n_timesteps=sequence_length+prediction_length

#generate a Sine dataset
Dataset_generator=BetaDataset(n_samples=n_samples, n_timesteps=n_timesteps)
X,ts,ys=Dataset_generator.get_X_ts_ys()

Ys=np.array(ys)
X_torch=torch.tensor(Ys[:,:sequence_length]).float().reshape(n_samples,sequence_length,1)
Y_torch=torch.tensor(Ys[:,sequence_length:]).float().reshape(n_samples,prediction_length,1)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

B=5
ts_N=[arr[sequence_length:] for arr in ts]
ys_N=[arr[sequence_length:] for arr in ys]

internal_knots=calculate_knot_placement(ts_N, ys_N, n_internal_knots=B-2, T=ts_N[0][-1], Ti=ts_N[0][0] ,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(ts_N[0][0],ts_N[0][-1]),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(ts)[:,sequence_length:]))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

torch.Size([900, 40, 1]) torch.Size([900, 20, 1])


100%|██████████| 10/10 [02:10<00:00, 13.01s/it]


Mean Test Loss: 0.0075, Std Test Loss: 0.0019


c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
100%|██████████| 10/10 [03:57<00:00, 23.75s/it]

Mean Test Loss: 0.0099, Std Test Loss: 0.0018


In [15]:
sequence_length = 40
prediction_length = 20
input_channels = 1
output_channels = 1
n_samples=2000
n_timesteps=sequence_length+prediction_length


#generate a Tumor dataset
args={
'n_samples': n_samples,'n_time_steps': n_timesteps,'time_horizon': 1.0,'noise_std': 0.0,'seed': 0,'equation': 'wilkerson'
}

Dataset_generator=SyntheticTumorDataset(**args)

X,ts,ys=Dataset_generator.get_X_ts_ys()

Ys=np.array(ys)
X_torch=torch.tensor(Ys[:,:sequence_length]).float().reshape(n_samples,sequence_length,1)
Y_torch=torch.tensor(Ys[:,sequence_length:]).float().reshape(n_samples,prediction_length,1)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

B=9
ts_N=[arr[sequence_length:] for arr in ts]
ys_N=[arr[sequence_length:] for arr in ys]

internal_knots=calculate_knot_placement(ts_N, ys_N, n_internal_knots=B-2, T=ts_N[0][-1], Ti=ts_N[0][0] ,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(ts_N[0][0],ts_N[0][-1]),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(ts)[:,sequence_length:]))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

torch.Size([2000, 40, 1]) torch.Size([2000, 20, 1])


100%|██████████| 10/10 [04:22<00:00, 26.28s/it]


Mean Test Loss: 0.0041, Std Test Loss: 0.0011


c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
100%|██████████| 10/10 [08:39<00:00, 51.97s/it]

Mean Test Loss: 0.0046, Std Test Loss: 0.0024


## Real datasets

In [ ]:
os.chdir('../experiments')
Dataset_generator=AirfoilDataset()
X,ts,ys=Dataset_generator.get_X_ts_ys()

sequence_length = 3
prediction_length = 9
input_channels = 1
output_channels = 1

Ys=np.array([y[:14] for y in ys if len(y)>=14])
Ts=np.array([t[:14] for t in ts if len(t)>=14])

X_torch=torch.tensor(Ys[:,:sequence_length]).float().reshape(-1,sequence_length,1)
Y_torch=torch.tensor(Ys[:,sequence_length:]).float().reshape(-1,prediction_length,1)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

B=9
ts_N=[arr[sequence_length:] for arr in ts]
ys_N=[arr[sequence_length:] for arr in ys]

internal_knots=calculate_knot_placement(ts_N, ys_N, n_internal_knots=B-2, T=1.0, Ti=0.0 ,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(0.0,1.0),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(Ts)[:,sequence_length:]))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

torch.Size([92, 3, 1]) torch.Size([92, 9, 1])


100%|██████████| 10/10 [00:10<00:00,  1.00s/it]
c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Mean Test Loss: 25.7765, Std Test Loss: 6.0219


100%|██████████| 10/10 [00:22<00:00,  2.24s/it]

Mean Test Loss: 31.7468, Std Test Loss: 12.7987


In [74]:
Dataset_generator=FLChainDataset()
X,ts,ys=Dataset_generator.get_X_ts_ys()

sequence_length = 10
prediction_length = 10
input_channels = 1
output_channels = 1

Ys=np.array( ys)
Ts=np.array(ts)

X_torch=torch.tensor(Ys[:,:sequence_length]).float().reshape(-1,sequence_length,1)
Y_torch=torch.tensor(Ys[:,sequence_length:]).float().reshape(-1,prediction_length,1)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

B=9
ts_N=[arr[sequence_length:] for arr in ts]
ys_N=[arr[sequence_length:] for arr in ys]

internal_knots=calculate_knot_placement(ts_N, ys_N, n_internal_knots=B-2, T=1.0, Ti=0.0 ,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(0.0,1.0),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(Ts)[:,sequence_length:]))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

torch.Size([5499, 10, 1]) torch.Size([5499, 10, 1])


100%|██████████| 10/10 [11:45<00:00, 70.57s/it]


Mean Test Loss: 0.0010, Std Test Loss: 0.0001


c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
100%|██████████| 10/10 [23:59<00:00, 143.96s/it]

Mean Test Loss: 0.0009, Std Test Loss: 0.0001


In [81]:
Dataset_generator=StressStrainDataset(**{
                "lot": "all",
                "include_lot_as_feature": True,
                "downsample": True,
                "more_samples": 0,
                "specimen": "all",
                "max_strain": 0.2})
X,ts,ys=Dataset_generator.get_X_ts_ys()

sequence_length = 140
prediction_length = 70
input_channels = 1
output_channels = 1

Ys=np.array([y[:210] for y in ys if len(y)>=210])
Ts=np.array([t[:210] for t in ts if len(t)>=210])

X_torch=torch.tensor(Ys[:,:sequence_length]).float().reshape(-1,sequence_length,1)
Y_torch=torch.tensor(Ys[:,sequence_length:]).float().reshape(-1,prediction_length,1)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

B=9
ts_N=[arr[sequence_length:] for arr in Ts]
ys_N=[arr[sequence_length:] for arr in Ys]

internal_knots=calculate_knot_placement(ts_N, ys_N, n_internal_knots=B-2, T=1 ,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(0,1),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(Ts)[:,sequence_length:]))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

torch.Size([47, 140, 1]) torch.Size([47, 70, 1])


100%|██████████| 10/10 [00:12<00:00,  1.24s/it]
c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Mean Test Loss: 0.0018, Std Test Loss: 0.0025


100%|██████████| 10/10 [00:16<00:00,  1.64s/it]

Mean Test Loss: 0.0018, Std Test Loss: 0.0026
